# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO"s and use more than one TODO code cell to do all your tasks.

In [1]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip -qq install smdebug

In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.pytorch import PyTorch

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
! wget -nc https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
! unzip -qq dogImages.zip

File ‘dogImages.zip’ already there; not retrieving.



In [10]:
! aws s3 sync /root/dog-breed-classification/dogImages s3://sagemaker-us-east-1-022926224205/dog-breed-project/data/

In [13]:
import os

os.listdir("/root/dog-breed-classification/dogImages/")

['test', 'valid', 'train']

the dataset already splitted to test, valid, and train

In [ ]:
! pip install -qq torchvision

In [21]:
import pandas as pd
import torchvision
from PIL import ImageFile

%matplotlib inline
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_dataset = torchvision.datasets.ImageFolder("~/dog-breed-classification/dogImages/train")
valid_dataset = torchvision.datasets.ImageFolder("~/dog-breed-classification/dogImages/valid")
test_dataset = torchvision.datasets.ImageFolder("~/dog-breed-classification/dogImages/test")

def get_class_distribution(dataset_obj):
    idx2class = {v: k for k, v in dataset_obj.class_to_idx.items()}
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    for _, label_id in dataset_obj:
        label = idx2class[label_id]
        count_dict[label] += 1
    return count_dict

In [17]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 6680
    Root location: /root/dog-breed-classification/dogImages/train

In [19]:
valid_dataset

Dataset ImageFolder
    Number of datapoints: 835
    Root location: /root/dog-breed-classification/dogImages/valid

In [20]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 836
    Root location: /root/dog-breed-classification/dogImages/test

In [50]:
print(f"total_class: {len(train_dataset.classes)}")
print(train_dataset.class_to_idx)

total_class: 133
{'001.Affenpinscher': 0, '002.Afghan_hound': 1, '003.Airedale_terrier': 2, '004.Akita': 3, '005.Alaskan_malamute': 4, '006.American_eskimo_dog': 5, '007.American_foxhound': 6, '008.American_staffordshire_terrier': 7, '009.American_water_spaniel': 8, '010.Anatolian_shepherd_dog': 9, '011.Australian_cattle_dog': 10, '012.Australian_shepherd': 11, '013.Australian_terrier': 12, '014.Basenji': 13, '015.Basset_hound': 14, '016.Beagle': 15, '017.Bearded_collie': 16, '018.Beauceron': 17, '019.Bedlington_terrier': 18, '020.Belgian_malinois': 19, '021.Belgian_sheepdog': 20, '022.Belgian_tervuren': 21, '023.Bernese_mountain_dog': 22, '024.Bichon_frise': 23, '025.Black_and_tan_coonhound': 24, '026.Black_russian_terrier': 25, '027.Bloodhound': 26, '028.Bluetick_coonhound': 27, '029.Border_collie': 28, '030.Border_terrier': 29, '031.Borzoi': 30, '032.Boston_terrier': 31, '033.Bouvier_des_flandres': 32, '034.Boxer': 33, '035.Boykin_spaniel': 34, '036.Briard': 35, '037.Brittany': 36, 

There're **133 classes** (dog breeds) with index 0 to 132

In [54]:
# get total sample of each class
class_dist = get_class_distribution(train_dataset)
class_dist

{'001.Affenpinscher': 64,
 '002.Afghan_hound': 58,
 '003.Airedale_terrier': 52,
 '004.Akita': 63,
 '005.Alaskan_malamute': 77,
 '006.American_eskimo_dog': 64,
 '007.American_foxhound': 50,
 '008.American_staffordshire_terrier': 66,
 '009.American_water_spaniel': 34,
 '010.Anatolian_shepherd_dog': 50,
 '011.Australian_cattle_dog': 66,
 '012.Australian_shepherd': 66,
 '013.Australian_terrier': 46,
 '014.Basenji': 69,
 '015.Basset_hound': 73,
 '016.Beagle': 59,
 '017.Bearded_collie': 62,
 '018.Beauceron': 50,
 '019.Bedlington_terrier': 48,
 '020.Belgian_malinois': 62,
 '021.Belgian_sheepdog': 64,
 '022.Belgian_tervuren': 47,
 '023.Bernese_mountain_dog': 65,
 '024.Bichon_frise': 62,
 '025.Black_and_tan_coonhound': 37,
 '026.Black_russian_terrier': 41,
 '027.Bloodhound': 64,
 '028.Bluetick_coonhound': 35,
 '029.Border_collie': 74,
 '030.Border_terrier': 52,
 '031.Borzoi': 56,
 '032.Boston_terrier': 65,
 '033.Bouvier_des_flandres': 45,
 '034.Boxer': 64,
 '035.Boykin_spaniel': 53,
 '036.Briar

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

lorem ipsum

In [55]:
role = sagemaker.get_execution_role()

#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "epochs": IntegerParameter(3, 5),
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "step_size": IntegerParameter(3, 5),
    "gamma": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
}

In [12]:
#TODO: Create estimators for your HPs
# TODO: Your estimator here
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    instance_type="ml.g4dn.xlarge",
    instance_count=1,
    framework_version="1.12",
    py_version="py38",
)

In [15]:
# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator=estimator,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=5,
    max_parallel_jobs=2,
    objective_metric_name="test accucary",
    objective_type="Maximize",
    metric_definitions=[{"Name": "test accucary", "Regex": "test_accuracy: ([0-9\\.]+)"}],
    early_stopping_type='Auto',
)

In [16]:
# TODO: Fit your HP Tuner
# TODO: Remember to include your data channels
tuner.fit({
    "train": "s3://sagemaker-us-east-1-022926224205/dog-breed-project/data/train/",
    "valid": "s3://sagemaker-us-east-1-022926224205/dog-breed-project/data/valid/",
    "test": "s3://sagemaker-us-east-1-022926224205/dog-breed-project/data/test/"
})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.........................................................................................................................................................................................................................................................................................................!


In [18]:
# TODO: Get the best estimators and the best HPs
#TODO
best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2023-01-10 08:53:23 Starting - Preparing the instances for training
2023-01-10 08:53:23 Downloading - Downloading input data
2023-01-10 08:53:23 Training - Training image download completed. Training in progress.
2023-01-10 08:53:23 Uploading - Uploading generated training model
2023-01-10 08:53:23 Completed - Resource reused by training job: pytorch-training-230110-0841-004-c4d1afde


{'_tuning_objective_metric': '"test accucary"',
 'batch_size': '"64"',
 'epochs': '4',
 'gamma': '0.05764456565718428',
 'learning_rate': '0.00968994167901433',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2023-01-10-08-41-44-936"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-022926224205/pytorch-training-2023-01-10-08-41-44-936/source/sourcedir.tar.gz"',
 'step_size': '4'}

In [53]:
# Get results from HPO
import pandas as pd

tuning_job_name = tuner.latest_tuning_job.name
tuner_results = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
# Get the results as a dataframe
full_df = tuner_results.dataframe()
##Display results sorted by objective metric values
if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=False)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 5
{'lowest': 0.7834929823875427, 'highest': 0.8122010231018066}


,batch_size,epochs,gamma,learning_rate,step_size,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
3,"""64""",4.0,0.057645,0.009690,4.0,pytorch-training-230110-0841-002-a6e5503d,Completed,0.812201,2023-01-10 08:42:57+00:00,2023-01-10 08:53:11+00:00,614.0
2,"""256""",4.0,0.061777,0.046865,4.0,pytorch-training-230110-0841-003-3e5583ae,Completed,0.801435,2023-01-10 08:52:10+00:00,2023-01-10 08:59:13+00:00,423.0
1,"""32""",5.0,0.002436,0.009374,4.0,pytorch-training-230110-0841-004-c4d1afde,Stopped,0.793062,2023-01-10 08:53:25+00:00,2023-01-10 09:00:18+00:00,413.0
4,"""32""",3.0,0.011719,0.013154,4.0,pytorch-training-230110-0841-001-08e643ba,Completed,0.789474,2023-01-10 08:42:58+00:00,2023-01-10 08:51:22+00:00,504.0
0,"""512""",4.0,0.007329,0.001621,3.0,pytorch-training-230110-0841-005-bfb0ceec,Stopped,0.783493,2023-01-10 08:59:34+00:00,2023-01-10 09:06:38+00:00,424.0


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()